# trajectory_compare

compare multiple particle trajectory files and .mat outputs with the same spawn points

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr

from utils import create_path
from parcels_utils import arrays_to_particlefilenc
from plot_utils import plot_trajectories

In [ ]:
def get_part_hash(ds, i):
    lat = ds["lat"].values[i, 0]
    lon = ds["lon"].values[i, 0]
    return hash((lat, lon))

In [ ]:
# first file in list is used as a reference for every particle's starting point
particle_file_paths = [
    "particledata/particle_plume_track.nc",
    "particledata/particle_plume_track_totsdlj.nc",
    "particledata/plume_pts_20200810_0030.nc",
]

particle_files = []
for path in particle_file_paths:
    with xr.open_dataset(path) as pf:
        particle_files.append(pf)

In [ ]:
ref_file = particle_files[0]

In [ ]:
particle_ids = np.empty(ref_file.dims["traj"])
for i in range(ref_file.dims["traj"]):
    particle_ids[i] = get_part_hash(ref_file, i)

iterates through the points in the reference netcdf and treats them as spawn points

then it checks all the other netcdf files for points with the same spawn, and then graphs them to compare

saves a separate netcdf file for each spawn and each different particle file given

In [ ]:
a = 250
for p_id in particle_ids[a:a+10]:
    nc_files = []
    for pf_i in range(len(particle_files)):
        pf = particle_files[pf_i]
        for i in range(pf.dims["traj"]):
            if p_id != get_part_hash(pf, i):
                continue
            num_format = format(p_id, ".0f")
            save_path = create_path(f"particledata/plume_compare/{num_format}")
            time = pf["time"].values[np.newaxis, i]
            lat = pf["lat"].values[np.newaxis, i]
            lon = pf["lon"].values[np.newaxis, i]
            # recreate particle netcdf to drop data we don't need
            ds = arrays_to_particlefilenc(time, lat, lon)
            name = particle_file_paths[pf_i].split("/")[-1].split(".")[0]
            ds.to_netcdf(f"{save_path / Path(name)}.nc")
            nc_files.append(f"{save_path / Path(name)}.nc")
            break
    plot_trajectories(nc_files, legend=True)